# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   635.037215 -0.794270  1234  634.242945  635.831485  1268.485890   
6   520.156135 -0.180829  1234  519.975306  520.336964  1039.950611   
7   795.342290  0.893088  1234  796.235378  794.449202  1592.470755   
8   664.109546  0.654381  1234  664.763928  663.455165  1329.527856   
9   635.636453  0.955410  1234  636.591863  634.681043  1273.183726   
10  246.337287  0.769750  1234  247.107037  245.567538   494.214073   
11  940.220179 -0.121430  1234  940.098749  940.341609  1880.197498   
12  696.057154  0.524155  1234  696.581309  695.532999  1393.162617   
13   67.012123 -0.578614  1234   66.433509   67.590738   132.867018   
14  861.652545 -0.416825  1234  861.235720  862.069370  1722.471439   

              f  
5   1271.662969  
6   1040.673929  
7   1588.898405  
8   1326.910330  
9   1269.362086  
10   491.135075  
11  1880.683217  
12  1391.065999  
13   135.181475  
14  1724.138741

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -634.242945   635.831485
6   -519.975306   520.336964
7   -796.235378   794.449202
8   -664.763928   663.455165
9   -636.591863   634.681043
10  -247.107037   245.567538
11  -940.098749   940.341609
12  -696.581309   695.532999
13   -66.433509    67.590738
14  -861.235720   862.069370

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -634.242945   635.831485
6   -519.975306   520.336964
7   -796.235378   794.449202
8   -664.763928   663.455165
9   -636.591863   634.681043
10  -247.107037   245.567538
11  -940.098749   940.341609
12  -696.581309   695.532999
13   -66.433509    67.590738
14  -861.235720   862.069370

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-634.24294476,  635.83148451],
       [-519.97530569,  520.33696434],
       [-796.2353777 ,  794.44920243],
       [-664.76392779,  663.455165  ],
       [-636.59186299,  634.68104324],
       [-247.10703674,  245.56753754],
       [-940.0987489 ,  940.34160864],
       [-696.58130865,  695.53299941],
       [ -66.43350907,   67.59073767],
       [-861.23571962,  862.06937037]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1266.485890  -1271.662969
6    1037.950611  -1040.673929
7    1590.470755  -1588.898405
8    1327.527856  -1326.910330
9    1271.183726  -1269.362086
10    492.214073   -491.135075
11   1878.197498  -1880.683217
12   1391.162617  -1391.065999
13    130.867018   -135.181475
14   1720.471439  -1724.138741

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf